In [1]:
import sys
sys.path.append('../')
from utils import paths
from utils import utils


import pandas as pd
import numpy as np


In [15]:
### We want to create the dataset, given the Location(lat, lon):

def prepare_data(location):
    '''
    location: name of location, in lowercase, e.g. 'aberdeen'.
    '''
    lat, lon = GetCoordinates(location)
    # Retrieve data from specified location in the UK
    
    # Perform data preprocessing or cleaning
    
    # Save prepared data to desired location


def GetCoordinates(location):
    '''
    location: name of location, in lowercase, e.g. 'aberdeen'.
    '''
    # Retrieve data from specified location in the UK
    # The metadata file is the paths.Haigh_meta_path
    df = pd.read_csv(paths.Haigh_meta_path)
    lat = df[df['location'] == location].latitude[0]
    lon = df[df['location'] == location].longitude[0]
    return lat, lon

def geo_coord_borders(lat, lon):
    """
    Get the borders of the window of interest for CMIP6 Native grid;
    To be passed down into extract_window function.
    ---
    lat (np.float): latitude of location
    lon (np.float): longitude of location
    ---
    Returns:
        ((lat_lw_bord,lat_up,bord),(lon_lw_bord,lon_up_bord)): (tuple(tuple(np.float)))
    """
    ### The CMIP6 N96 Native grid has   144 gridcells of latitude, i.e. 1.25deg/cell, and
    ###                                 192 gridcells of longitude, i.e. 1.875deg/cell.

    ### Make sure the longitude is in [-180,180], not [0,360]:
    if (lon - 180 >0):
        lon = lon - 360

    delta_lat = 1.25 * 2.5
    delta_lon = 1.875* 2.5

    lat_lw_bord = lat - delta_lat 
    lat_up_bord = lat + delta_lat

    lon_lw_bord = lon - delta_lon
    lon_up_bord = lon + delta_lon

    if ((lon_lw_bord >= 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon
        lon_up_bord = lon + delta_lon
    elif ((lon_lw_bord < 0)&(lon_up_bord <= 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon + 360
    elif ((lon_lw_bord < 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon
    else:
        print("Error: Something is wrong with the Longitude.")

    return ((lat_lw_bord,lat_up_bord),(lon_lw_bord,lon_up_bord))

#--------------------------------------------------------------------------------------------------
def extract_window(data,geo_coord_tuple,data_type = 'CMIP6'):
    """
    data_type (string): 'CMIP6' or 'ERA5'
    THIS FUNCTION FAILS FOR NEGATIVE LONGITUDES ---- FIX!
    """


    if data_type == 'CMIP6':

        lat_lw = geo_coord_tuple[0][0]
        lat_up = geo_coord_tuple[0][1]
        lon_lw = geo_coord_tuple[1][0]
        lon_up = geo_coord_tuple[1][1]

        data = data.sel(lat = slice(lat_lw,lat_up))

        ### Check if the window crosses the meridian:
        if ((lon_lw >180)&(lon_lw<360))&((lon_up>0)&(lon_up<180)):
            crop1 = data.sel(lon = slice(lon_lw,360))
            crop2 = data.sel(lon = slice(0,lon_up))
            data = xr.concat([crop1,crop2],dim = 'lon')
        else:
            data = data.sel(lon = slice(lon_lw,lon_up))
    elif data_type == 'ERA5':
        ### ERA5 data has longitude in [-180,180]
        lat_lw = geo_coord_tuple[0][1]
        lat_up = geo_coord_tuple[0][0]
        lon_lw = geo_coord_tuple[1][0]
        lon_up = geo_coord_tuple[1][1]

        data = data.sel(latitude = slice(lat_lw,lat_up))

        ### Check if the window crosses the meridian:
        if ((lon_lw >180)&(lon_lw<360))&((lon_up>0)&(lon_up<180)):
            crop1 = data.sel(longitude = slice(lon_lw-360,360))
            crop2 = data.sel(longitude = slice(0,lon_up))
            data = xr.concat([crop1,crop2],dim = 'longitude')
        elif ((lon_lw >180)&(lon_lw<360))&((lon_up>180)&(lon_up<360)):
            data = data.sel(longitude = slice(lon_lw-360,lon_up-360))
        else:
            data = data.sel(longitude = slice(lon_lw,lon_up))
    else:
        print('data_type not recognized.')

    return data

In [2]:
uk_haigh_meta = pd.read_csv(paths.Haigh_meta_path).drop(['Unnamed: 0','index'],axis=1)
uk_haigh_meta.head()

,location,latitude,longitude,file_name,geometry
0,aberdeen,57.14325,-2.07451,aberdeen-p038-uk-bodc,POINT (-2.07451 57.14325)
1,avonmouth,51.51089,-2.71497,avonmouth-p060-uk-bodc,POINT (-2.71497 51.51089)
2,bangor,54.66475,-5.66947,bangor-p662-uk-bodc,POINT (-5.66947 54.66475)
3,barmouth,52.71906,-4.04517,barmouth-p923-uk-bodc,POINT (-4.04517 52.71906)
4,bournemouth,50.71433,-1.87486,bournemouth-p988-uk-bodc,POINT (-1.87486 50.71433)


In [4]:
storms = pd.read_csv(paths.storms_path, skiprows = 2, header = 3)
water_level= pd.read_csv(paths.water_levels_path,skiprows = 2, header =3)

In [17]:
lat, lon = GetCoordinates('aberdeen')
geo_coord_borders(lat, lon)

((54.01825, 60.26825), (353.23799, 2.61299))

In [18]:
lat_lw = lat - 3.125
lat_up = lat + 3.125
lon_lw = lon - 3.125
lon_up = lon + 3.125
geo_borders = ((lat_lw,lat_up),(lon_lw,lon_up))

In [19]:
geo_borders

((54.01825, 60.26825), (-5.19951, 1.05049))